<a href="https://colab.research.google.com/github/Cereha/Statistical-methods-of-recognition/blob/main/Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import scipy.stats as stats

In [2]:
# Функція генератор
def generate_data(n, P_k1, P_k2, a1, a2, sigma):
    data = np.zeros(n)
    states = np.random.choice([1, 2], size=n, p=[P_k1, P_k2])

    for i, state in enumerate(states):
        if state == 1:
            data[i] = np.random.normal(loc=a1, scale=sigma)
        else:
            data[i] = np.random.normal(loc=a2, scale=sigma)

    return data, states

In [3]:
# Ініціалізуємо параметри
P_k1, P_k2 = 0.5, 0.5
a1, a2 = 1, 2
sigma = 1
n = 100

# Генеруємо дані
data, true_states = generate_data(n, P_k1, P_k2, a1, a2, sigma)

In [4]:
# Функція ймовірності кроку
def expectation_step(data, P_k1, P_k2, a1, a2, sigma):
    pdf_state1 = stats.norm.pdf(data, loc=a1, scale=sigma)
    pdf_state2 = stats.norm.pdf(data, loc=a2, scale=sigma)

    # Віднормуємо ймовірності
    norm = P_k1 * pdf_state1 + P_k2 * pdf_state2

    # Обчислюємо ймовірності кожного стану для кожного спостереження
    gamma_k1 = P_k1 * pdf_state1 / norm
    gamma_k2 = P_k2 * pdf_state2 / norm

    return gamma_k1, gamma_k2

In [5]:
#функція максимізації
def maximization_step(data, gamma_k1, gamma_k2):
    # Оцінка нових параметрів
    P_k1 = np.mean(gamma_k1)
    P_k2 = np.mean(gamma_k2)

    a1 = np.sum(gamma_k1 * data) / np.sum(gamma_k1)
    a2 = np.sum(gamma_k2 * data) / np.sum(gamma_k2)

    sigma1 = np.sqrt(np.sum(gamma_k1 * (data - a1)**2) / np.sum(gamma_k1))
    sigma2 = np.sqrt(np.sum(gamma_k2 * (data - a2)**2) / np.sum(gamma_k2))

    # Обновлення параметрів
    sigma = (sigma1 + sigma2) / 2

    return P_k1, P_k2, a1, a2, sigma

In [6]:
#Алгоритм самонавчання
def self_learning_algorithm(data, P_k1, P_k2, a1, a2, sigma, tolerance=0.001, max_iterations=100):
    for iteration in range(max_iterations):
        # E-step
        gamma_k1, gamma_k2 = expectation_step(data, P_k1, P_k2, a1, a2, sigma)

        # M-step
        P_k1_new, P_k2_new, a1_new, a2_new, sigma_new = maximization_step(data, gamma_k1, gamma_k2)

        # Перевірка зупинки
        if np.abs(P_k1_new - P_k1) < tolerance and np.abs(P_k2_new - P_k2) < tolerance \
                and np.abs(a1_new - a1) < tolerance and np.abs(a2_new - a2) < tolerance \
                and np.abs(sigma_new - sigma) < tolerance:
            break

        # Оновлення параметрів
        P_k1, P_k2, a1, a2, sigma = P_k1_new, P_k2_new, a1_new, a2_new, sigma_new

    return P_k1, P_k2, a1, a2, sigma


In [7]:
learned_P_k1, learned_P_k2, learned_a1, learned_a2, learned_sigma = self_learning_algorithm(data, P_k1, P_k2, a1, a2, sigma)

# Виводимо отримані результати
print("Навчені параметри:")
print("P(k=1):", learned_P_k1)
print("P(k=2):", learned_P_k2)
print("a1:", learned_a1)
print("a2:", learned_a2)
print("Дисперсія:", learned_sigma)

Навчені параметри:
P(k=1): 0.5901670112611836
P(k=2): 0.40983298873881624
a1: 1.0784820223931282
a2: 2.116838335437074
Дисперсія: 1.0893740273891197
